<a href="https://colab.research.google.com/github/AishwaryaHoysal24/PyTorch-Zero-to-GANs/blob/Main/Linear_Reg_using_PyTorch_built_ins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [6]:
import torch.nn as nn
import numpy as np
import torch

As before, we represent the inputs and targets and matrices.

In [7]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [8]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [10]:
from torch.utils.data import TensorDataset

In [13]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [14]:
from torch.utils.data import DataLoader

In [15]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [16]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 74.,  66.,  43.],
        [103.,  43.,  36.],
        [ 87., 135.,  57.],
        [ 92.,  87.,  64.]])
tensor([[ 22.,  37.],
        [ 57.,  69.],
        [ 20.,  38.],
        [118., 134.],
        [ 82., 100.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

In [17]:
# Define model
model = nn.Linear(3, 2) #Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2577, -0.5598, -0.5693],
        [ 0.0657, -0.4802,  0.4737]], requires_grad=True)
Parameter containing:
tensor([ 0.0919, -0.2500], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [18]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2577, -0.5598, -0.5693],
         [ 0.0657, -0.4802,  0.4737]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0919, -0.2500], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [19]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -80.7123,   -7.2557],
        [-109.0638,   -6.2092],
        [-130.3687,  -31.4015],
        [ -71.3349,    3.3318],
        [-111.2882,   -8.6541],
        [ -80.4102,   -6.7099],
        [-109.0733,   -5.2553],
        [-131.1957,  -30.8621],
        [ -71.6370,    2.7859],
        [-111.5998,   -8.2462],
        [ -80.7218,   -6.3019],
        [-108.7617,   -5.6633],
        [-130.3591,  -32.3553],
        [ -71.0233,    2.9238],
        [-111.5902,   -9.2000]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.


In [20]:
# Import nn.functional
import torch.nn.functional as F #The `nn.functional` package contains many useful loss functions and several other utilities.

In [21]:
# Define loss function
loss_fn = F.mse_loss

In [22]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(23321.9492, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [23]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [24]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb,yb in train_dl:

            # 1. Generate predictions
            pred = model(xb)

            # 2. Calculate loss
            loss = loss_fn(pred, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()

        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [25]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 593.6873
Epoch [20/100], Loss: 467.0055
Epoch [30/100], Loss: 251.4763
Epoch [40/100], Loss: 303.7090
Epoch [50/100], Loss: 222.6546
Epoch [60/100], Loss: 127.5101
Epoch [70/100], Loss: 62.4735
Epoch [80/100], Loss: 10.8209
Epoch [90/100], Loss: 52.5740
Epoch [100/100], Loss: 21.8250


In [26]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.6312,  71.6826],
        [ 79.5755, 101.6549],
        [121.0948, 128.5493],
        [ 30.1163,  45.2157],
        [ 92.1782, 116.0134],
        [ 57.5375,  70.8341],
        [ 78.8891, 101.8509],
        [121.1496, 129.2678],
        [ 31.2099,  46.0641],
        [ 92.5855, 117.0579],
        [ 57.9448,  71.8786],
        [ 78.4818, 100.8064],
        [121.7812, 128.3533],
        [ 29.7090,  44.1712],
        [ 93.2719, 116.8619]], grad_fn=<AddmmBackward0>)

In [27]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [ ]:
model(torch.tensor([[75, 63, 44.]]))


The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.

In [28]:
!pip install jovian --upgrade -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.0 MB/s eta 0:00:00


In [29]:
import jovian

In [31]:
jovian.commit(project='03-linear-regression')

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.
